<a href="https://colab.research.google.com/github/Abdulazizbek/Python_code_tips_all/blob/master/Keras_mnist_DCGAN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [0]:
discriminator = keras.Sequential([
                                  keras.Input(shape=(28, 28, 1)),
                                  layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'),
                                  layers.LeakyReLU(alpha=0.2),
                                  layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'),
                                  layers.LeakyReLU(alpha=0.2),
                                  layers.GlobalMaxPooling2D(),
                                  layers.Dense(1)
                                  ], name='Discriminator')

In [0]:
discriminator.summary()

Model: "Discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 74,625
Trainable params: 74,625
Non-trainable params: 0
_________________________________________________

In [0]:
latent_dim = 128

generator = keras.Sequential([
                              keras.Input(shape=(latent_dim, )),
                              layers.Dense(7 * 7 * 128),
                              layers.LeakyReLU(alpha=0.2),
                              layers.Reshape((7, 7, 128)),
                              layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'),
                              layers.LeakyReLU(alpha=0.2),
                              layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'),
                              layers.LeakyReLU(alpha=0.2),
                              layers.Conv2D(1, (7, 7), padding='same', activation='sigmoid')
], name='Generator')

In [0]:
generator.summary()

Model: "Generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 6272)              809088    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 6272)              0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 28, 28, 128)       0 

In [0]:
d_optimizer = keras.optimizers.Adam(learning_rate=0.0003)
g_optimizer = keras.optimizers.Adam(learning_rate=0.0004)

#loss function
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)

In [0]:
@tf.function
def train_step(real_images):
  # Sample random points in the latent space
  random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))

  # decode them to fake images
  generated_images = generator(random_latent_vectors)

  # Combine them with real images
  combined_images = tf.concat([generated_images, real_images], axis=0)
  
  # Assemble  labels discriminating real from fake images
  labels = tf.concat([tf.ones((batch_size, 1)),
                      tf.zeros((real_images.shape[0], 1))], axis=0)
  
  # Add random noise to the labels - important trick
  labels += 0.05 * tf.random.uniform(labels.shape)

  # Train the discriminator
  with tf.GradientTape() as tape:
    predictions = discriminator(combined_images)
    d_loss = loss_fn(labels, predictions)

  grads = tape.gradient(d_loss, discriminator.trainable_weights)
  d_optimizer.apply_gradients(zip(grads, discriminator.trainable_weights))

  # Sample random points in the latent space
  random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))

  #Assemble labels that say "all real images"
  misleading_labels = tf.zeros(batch_size, 1)

  # Train the generator (we are not updating weights of discriminator)
  with tf.GradientTape() as tape:
    predictions = discriminator(generator(random_latent_vectors))
    g_loss = loss_fn(misleading_labels, predictions)
  grads = tape.gradient(g_loss, generator.trainable_weights)
  g_optimizer.apply_gradients(zip(grads, generator.trainable_weights))
  return d_loss, g_loss, generated_images

In [0]:
# Prepare the dataset. We are using both training and test MNIST digits
batch_size = 64
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_digits = all_digits.astype('float32') / 255.
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))

dataset = tf.data.Dataset.from_tensor_slices(all_digits)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

In [0]:
epochs = 100
save_dir = './'

for epoch in range(epochs):
  print('\n Start epoch', epoch)

  for step, real_images in enumerate(dataset):
    # Train the discriminator & generator on one batch of real images
    d_loss, g_loss, generated_images = train_step(real_images)

    # Occasionaly save / plot
    if step % 200 == 0:
      print('discriminator loss at step %d: %.4f' % (step, d_loss))
      print('generator loss at step %d: %.4f' % (step, g_loss))

      # Save one generated image
      img = tf.keras.preprocessing.image.array_to_img(
          generated_images[0] * 255., scale=False)
      img.save(os.path.join(save_dir, 'generated_img' + str(step) + '.png'))


 Start epoch 0
discriminator loss at step 0: 0.7159
generator loss at step 0: 0.7522
discriminator loss at step 200: 0.5561
generator loss at step 200: 1.1286
discriminator loss at step 400: 0.5207
generator loss at step 400: 1.4344
discriminator loss at step 600: 0.5691
generator loss at step 600: 1.5948
discriminator loss at step 800: 0.5590
generator loss at step 800: 1.3196
discriminator loss at step 1000: 0.2967
generator loss at step 1000: 1.8278

 Start epoch 1
discriminator loss at step 0: 0.2634
generator loss at step 0: 2.1113
discriminator loss at step 200: 0.2040
generator loss at step 200: 2.3185
discriminator loss at step 400: 0.1500
generator loss at step 400: 3.2303
discriminator loss at step 600: 0.1795
generator loss at step 600: 3.0537
discriminator loss at step 800: 0.1007
generator loss at step 800: 2.9257
discriminator loss at step 1000: 0.1062
generator loss at step 1000: 3.1215

 Start epoch 2
discriminator loss at step 0: 0.0936
generator loss at step 0: 4.069